# Pseudo Code: Smart Budget Allocation Algorithm

## Step 1: Load Dataset
- Read the CSV file containing expense data into a DataFrame.

## Step 2: Initialize Variables
- Create lists for `OverBudget` and `UnderBudget` categories.
- Ask the user for:
  - Total monthly budget.
  - Individual budgets for categories like Transport, Food, Entertainment, School Expenses, and Other.

## Step 3: Filter Data by Categories
- For each expense type (e.g., Transport, Food), filter the dataset into separate DataFrames.

## Step 4: Classify Expenses as Over-Budget or Under-Budget
- Loop through the filtered DataFrames:
  - Calculate total spending in each category.
  - Compare with the allocated budget.
  - Append to `OverBudget` if spending exceeds the budget.
  - Append to `UnderBudget` if spending is within the budget.

## Step 5: Calculate Average Daily Budgets
- For each category:
  - Calculate the total spending.
  - Divide by the number of unique days to get the average daily budget.

## Step 6: Analyze Expenses for Each Date
- Get a list of all unique dates in the dataset.

### Loop Through Each Date
#### Step 6.1: Analyze Over-Budget Categories
- For each over-budget category:
  - Filter expenses for the current date.
  - Calculate total spending.
  - Compare total spending with the daily average budget.
  - Add the excess amount (if any) to `over_budget_total`.

#### Step 6.2: Analyze Under-Budget Categories
- For each under-budget category:
  - Filter expenses for the current date.
  - Calculate total spending.
  - Compare with the daily average budget.
  - Calculate remaining budget (if any).
  - Append the category and remaining budget to `under_budget_allocation`.

#### Step 6.3: Reallocate Funds
- If there are under-budget categories:
  - Calculate the total reallocatable budget.
  - Distribute `over_budget_total` proportionally among under-budget categories.
  - Print reallocation suggestions.
- Else:
  - Print a message advising the user to reduce spending in over-budget categories.

## Step 7: Identify High-Value Expenses
- For each over-budget category and date:
  - Sort expenses in descending order by amount.
  - Display the top 3 highest expenses to suggest reductions.





In [ ]:
import pandas as pd

# Load the dataset from a CSV file
data = pd.read_csv("assets/November_Budget_Expenditure.csv")

# Initialize lists to store over-budget and under-budget datasets
OverBudget = []
UnderBudget = []

# Collect user inputs for total monthly budget and individual category budgets
total_budget = int(input("What is your total budget for this month?"))
tranposport_budget = int(input("What is your budget for transport?"))
food_budget = int(input("What is your budget for food?"))
entertainment_budget = int(input("What is your budget for entertainment?"))
school_expenses_budget = int(input("What is your budget for school expenses?"))
other_budget = int(input("What is your budget for other expenses?"))

# Filter the dataset by expense types into separate DataFrames
transport = data[data["Type"] == "Transport"]            # Transport expenses
food = data[data["Type"] == "Food"]                      # Food expenses
entertainment = data[data["Type"] == "Entertainment"]    # Entertainment expenses
school_expenses = data[data["Type"] == "School expenses"] # School-related expenses
other = data[data["Type"] == "Other"]                    # Miscellaneous expenses

# Combine the datasets and budgets into lists for easy iteration
datasets = [tranport, food, entertainment, school_expenses, other]
budget = [tranport_budget, food_budget, entertainment_budget, school_expenses_budget, other]

# Classify each dataset as over-budget or under-budget
for index in range(len(datasets)):
    data_type = datasets[index]  # Select the current dataset (e.g., Transport, Food)
    data_budget = budget[index]  # Select the corresponding budget
    total_spent = data_type["Amount"].sum()  # Calculate total spent in this category

    # Append to OverBudget or UnderBudget based on the total spent vs budget
    OverBudget.append(data_type) if total_spent > data_budget else UnderBudget.append(data_type)

# Calculate the average daily budget for each category
# This will help analyze daily spending patterns against the allocated budget
average_budgets = {
    name: data_type["Amount"].sum() / len(data_type["Day"].unique())  # Average daily spend
    for name, data_type in zip(
        ["Transport", "Food", "Entertainment", "School Expenses", "Other"],
        OverBudget + UnderBudget
    )
}

# The resulting `average_budgets` dictionary contains:
# - Keys: Category names (e.g., "Transport", "Food")
# - Values: Average daily spending for each category

# Example usage: Display the average daily budgets for presentation
print("Average daily budgets for each category:")
for category, avg_budget in average_budgets.items():
    print(f"{category}: ${avg_budget:.2f}")



FileNotFoundError: [Errno 2] No such file or directory: 'November_Budget_Expenditure.csv'

In [ ]:
# List of unique dates where expenses will be analyzed for allocation
dates_for_allocation = data_type["Date"].unique().tolist()

# Iterate through each date in the allocation list
for date in dates_for_allocation:
    # Initialize variables to track over-budget totals and possible reallocation opportunities
    over_budget_total = 0  # Tracks the amount exceeding the daily budget across all over-budget categories
    under_budget_allocation = []  # Stores under-budget categories and their remaining budget for reallocation

    # Analyze over-budget categories for the current date
    for data_type in OverBudget:
        # Filter expenses for the current date
        expenses_on_date = data_type[data_type["Date"] == date]
        # Calculate the total expenses for this category on the current date
        total_expenses = expenses_on_date["Amount"].sum()
        # Calculate the daily average budget for this category
        average_budget_per_day = data_type["Amount"].sum() / len(data_type["Date"].unique())
        # If the daily expenses exceed the average budget, calculate the excess
        if total_expenses > average_budget_per_day:
            over_budget_total += total_expenses - average_budget_per_day

    # Compare the over-budget date with under-budget categories
    for data_type in UnderBudget:
        # Filter expenses for the current date
        expenses_on_date = data_type[data_type["Date"] == date]
        # Calculate the total expenses for this category on the current date
        total_expenses = expenses_on_date["Amount"].sum()
        # Calculate the daily average budget for this category
        average_budget_per_day = data_type["Amount"].sum() / len(data_type["Date"].unique())
        # If the daily expenses are under the average budget, calculate the remaining budget
        if total_expenses < average_budget_per_day:
            remaining_budget = average_budget_per_day - total_expenses
            # Store the under-budget category and its remaining budget
            under_budget_allocation.append((data_type, remaining_budget))

    # Reallocate funds proportionally across under-budget categories
    if under_budget_allocation:
        # Calculate the total amount available for reallocation
        total_reallocatable_budget = sum(x[1] for x in under_budget_allocation)
        # Distribute the over-budget total proportionally among under-budget categories
        for data_type, remaining_budget in under_budget_allocation:
            # Calculate the proportion of the remaining budget relative to the total reallocatable budget
            proportion = remaining_budget / total_reallocatable_budget
            # Calculate the amount to reallocate to this category
            reallocated_amount = proportion * over_budget_total
            # Display reallocation suggestion
            print(f"Reallocate ${reallocated_amount:.2f} from {data_type['Type'].iloc[0]} budget for {date}.")
    else:
        # If no under-budget categories are available, suggest reducing spending
        print(f"Reduce spending by ${over_budget_total:.2f} on {date} for over-budget categories.")



